In [1]:
# import libraries
import datetime
import string
import seaborn as sns
import numpy as np
from numpy import log as ln, array, exp

import gensim
from gensim import corpora, models, similarities,matutils
import random
from scipy.special import softmax
import os

from collections import namedtuple, Counter, defaultdict
import time
import math
from matplotlib import pyplot
import json

# from generative_model import HDHProcess
# from smc import Particle, infer

from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding

In [2]:
def preprocess(data_dir, save_dir):
    corpus_unprocessed = corpora.MmCorpus(data_dir + '/corpus.mm')
    dictionary = corpora.Dictionary.load(data_dir + '/dictionary.dict')
    views_ind_unprocessed = np.load(data_dir + '/views_ind_1.npy')
    temp_dict = {}
    for i,v in enumerate(views_ind_unprocessed):
        temp_dict[dictionary[i]] = v
    all_docs = []
    len_doc = []
    for i, doc in enumerate(corpus_unprocessed):
        words = doc
        #print words
        new_words = []
        for w in words:
            for j in range(int(w[1])):
                new_words.append(dictionary[w[0]])
        all_docs.append(new_words)
        len_doc.append(len(new_words))
    entities_frequency = list(dictionary.dfs.values())
    hand_picked_stop_words = ['http', 'https', 'www']
    del_ids_stopWords = [dictionary.token2id[e] for i, e in enumerate(hand_picked_stop_words)]
    del_ids_lowFreqs = [i for i, e in enumerate(entities_frequency) if e <= 0]  # or e >= 2000
    del_ids = list(set(del_ids_stopWords) | set(del_ids_lowFreqs))
    #del_ids = list()
    #print('removing terms, length dictionary before filter: ', len(del_ids), len(dictionary))
    #dictionary.filter_tokens(bad_ids=del_ids)
    #print('length dictionary after filter: ', len(dictionary))
    # " check the entities in e.g. people. The view_index for each category: People view 3 , Application view 2, KW view 1, BOW view 0 "
    # people_ind = [i for i in range(len(views_ind_unprocessed)) if views_ind_unprocessed[i] == 3]
    # people = []
    # for i in range(len(people_ind)):
    #     people.append(dictionary[people_ind[i]])
    #views_ind = np.delete(views_ind_unprocessed, del_ids)
    views_ind = np.zeros((len(dictionary),), dtype=int)
    for key in dictionary:
        views_ind[key] = temp_dict[dictionary[key]]
    print('num of terms unprocessed: ', corpus_unprocessed.num_terms)
    corpus_processed = [dictionary.doc2bow(doc) for doc in all_docs]
    #corpus_processed = corpus_unprocessed
    
    dictionary.save(save_dir + 'dictionary_processed.dict')
    corpora.MmCorpus.serialize(save_dir + 'corpus_processed.mm', corpus_processed)
    corpus = corpora.MmCorpus(save_dir + 'corpus_processed.mm')
    print('num of terms processed: ', corpus.num_terms)
    print('views_ind leng: ', len(views_ind))
    #views_ind = views_ind_unprocessed
    vocab_size = corpus.num_terms

    all_docs = []
    len_doc = []
    for i, doc in enumerate(corpus):
        words = doc
        words = [dictionary[w[0]] for w in words]
        all_docs.append(words)
        len_doc.append(len(words))
    return(corpus, dictionary, views_ind, vocab_size)

In [3]:
# A DataLoader is an object that loads the log data
# A data matrix is a matrix with n rows (num_data) and d columns (num_features)
# the features are themselves grouped in different views (num_views)
class DataLoader:
    def __init__(self, data_dir, save_dir):
        """For initialization"""
        #Parameters

        self.corpus, self.dictionary, self.views_ind, self.vocab_size = preprocess(data_dir, save_dir)

        self.num_features = self.corpus.num_terms          #total number of features # here num_features is num_entities
        self.num_data = self.corpus.num_docs               #total number of data (snapshots)
        self.num_views = max(self.views_ind) + 1           #total number of views views= [0=BOW, 1=KW, 2=App, 3=People]
        self.Data = None                                   #todo: (it may be impossible to create this) a num_data*num_featurs array
        #name of the features
        self.feature_names = [self.dictionary.get(i) for i in range(self.num_features) ]
        self.num_items_per_view = [sum(self.views_ind == i) for i in range(self.num_views)]

    def print_info(self):
        print ('The corpus has %d items' %self.num_data+' and %d features'%self.num_features+\
              ' and %d views' %self.num_views +' there are %d' %self.corpus.num_nnz +' non-zero elements')
        print ('People view %d' % self.num_items_per_view[3]+ ' items, Application view %d' %self.num_items_per_view[2]+\
              ' items, Doc view %d' %self.num_items_per_view[4]+\
              ' items, KW view %d' %self.num_items_per_view[1]+' items, BOW view %d' %self.num_items_per_view[0]+' items.')

    def process_item_info(self):
        #This function is used for offline feedback gathering
        print ('The corpus has %d items' %self.num_data+' and %d features'%self.num_features+\
              ' and %d views' %self.num_views +' there are %d' %self.corpus.num_nnz +' non-zero elements')
        print ('People view %d' %sum(self.views_ind == 3)+ ' items, Application view %d' %sum(self.views_ind == 2)+\
              ' items, Doc view %d' %sum(self.views_ind == 4)+\
              ' items, KW view %d' %sum(self.views_ind == 1)+' items, BOW view %d' %sum(self.views_ind == 0)+' items.')

        def returnReverse(k,v):
            return (v,k)
        #get the document frequency of the terms (i.e. how many document did a particular term occur in):
        term_frequency_dic = self.dictionary.dfs
        sorted_term_ferequency = sorted(term_frequency_dic.items(), key=lambda x : x[1], reverse=True)
        sorted_IDs = [sorted_term_ferequency[i][0] for i in range(self.num_features)]

        count_term = [y for (x,y) in sorted_term_ferequency]
        pyplot.hist(count_term[9000:self.num_features-1], 20, facecolor='green')
        pyplot.xlabel('number of occurrences in the corpus')
        pyplot.ylabel('count')
        pyplot.show()
        num_of_1_occurance = len([y for (x,y) in sorted_term_ferequency if y==1])
        print ('%d terms' %num_of_1_occurance+' have only appeared once in the corpus')
        term_names_1_occurance = [(self.feature_names[x]) for (x,y) in sorted_term_ferequency if y==1 ]
        #with open('term_names_1_occurance.txt', 'w') as outfile:
        #    json.dump(term_names_1_occurance, outfile)
        #those terms can be removed from the dictionary.todo: HOWEVER, they should be removed when the corpus is being made
        #print self.dictionary
        #self.dictionary.filter_extremes(no_below=2)
        #print self.dictionary

        AP_names = [(self.feature_names[sorted_IDs[i]]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 2]
        AP_ids = [(sorted_IDs[i]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 2]
        
        Doc_names = [(self.feature_names[sorted_IDs[i]]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 4]
        Doc_ids = [(sorted_IDs[i]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 4]

        KW_names = [(self.feature_names[sorted_IDs[i]]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 1]
        KW_ids = [(sorted_IDs[i]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 1]

        People_names = [(self.feature_names[sorted_IDs[i]]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 3]
        People_ids = [(sorted_IDs[i]) for i in range(self.num_features) \
                    if  self.views_ind[sorted_IDs[i]] == 3]

        num_to_show = 1000 #
        data = {}
        data["AP_names"] = AP_names[:num_to_show]
        data["Doc_names"] = Doc_names[:num_to_show]
        data["KW_names"] = KW_names[:num_to_show]
        data["People_names"] = People_names[:num_to_show]
        data["AP_ids"] = AP_ids[:num_to_show]
        data["Doc_ids"] = Doc_ids[:num_to_show]
        data["KW_ids"] = KW_ids[:num_to_show]
        data["People_ids"] = People_ids[:num_to_show]

        print(data)

In [4]:
# Load data
data = DataLoader('data/test', 'tmp/')
data.print_info()

num of terms unprocessed:  67951
num of terms processed:  67951
views_ind leng:  67951
The corpus has 5460 items and 67951 features and 5 views there are 511910 non-zero elements
People view 62 items, Application view 188 items, Doc view 1034 items, KW view 44450 items, BOW view 16757 items.


In [114]:
# doc title from file, can ignore later on
import io
_docs = []
_to_print = {}
for d in data.corpus:
    ts = ''
    app = ''
    title = ''
    for w in d:
        if data.views_ind[w[0]] == -1:
            ts = data.dictionary.get(w[0]).split('ts_')[1]
        if data.views_ind[w[0]] == 2:
            app = data.dictionary.get(w[0])
        if data.views_ind[w[0]] == 4:
            title = data.dictionary.get(w[0])
    _docs.append((ts,app,title))
    _to_print[ts]=[title,app]

137/137 [==============================] - 13s 93ms/step - loss: 0.0068 - accuracy: 0.2546 - val_loss: 0.0060 - val_accuracy: 0.3464
Epoch 5/100
137/137 [==============================] - 12s 86ms/step - loss: 0.0067 - accuracy: 0.2740 - val_loss: 0.0059 - val_accuracy: 0.4295
Epoch 6/100
137/137 [==============================] - 12s 86ms/step - loss: 0.0066 - accuracy: 0.2933 - val_loss: 0.0058 - val_accuracy: 0.4545
Epoch 7/100
137/137 [==============================] - 11s 82ms/step - loss: 0.0065 - accuracy: 0.3141 - val_loss: 0.0057 - val_accuracy: 0.4625
Epoch 8/100
136/137 [============================>.] - ETA: 0s - loss: 0.0064 - accuracy: 0.3180WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 5s 33ms/step - loss: 0.0064 - accuracy: 0.3193 - val_loss: 0.0057 - val_accuracy: 0.4554
Epoch 9/100
137/137 [==============================] - 4s 31ms/step - loss: 0.0064 - accuracy: 0.3344 - val_loss: 0.0057 - val_accuracy: 0.4616
Epoch 10/100
137/137 [==============================] - 4s 31ms/step - loss: 0.0063 - accuracy: 0.3328 - val_loss: 0.0056 - val_accuracy: 0.4625
Epoch 11/100
137/137 [==============================] - 4s 27ms/step - loss: 0.0063 - accuracy: 0.3394 - val_loss: 0.0056 - val_accuracy: 0.4625
Epoch 12/100
137/137 [==============================] - 12s 87ms/step - loss: 0.0063 - accuracy: 0.3380 - val_loss: 0.0056 - val_accuracy: 0.4634
Epoch 13/100
136/137 [============================>.] - ETA: 0s - loss: 0.0062 - accuracy: 0.3543INFO:tensorflow:Assets written to: ./checkpoints/LSTM_LDA-epoch013-topics131-sequence10-loss0.0062-acc0.3526-val_loss0.0056-val_acc0.4661/assets


137/137 [==============================] - 19s 142ms/step - loss: 0.0062 - accuracy: 0.3526 - val_loss: 0.0056 - val_accuracy: 0.4661
Epoch 14/100
137/137 [==============================] - 5s 33ms/step - loss: 0.0063 - accuracy: 0.3499 - val_loss: 0.0056 - val_accuracy: 0.4661
Epoch 15/100
137/137 [==============================] - 4s 28ms/step - loss: 0.0062 - accuracy: 0.3504 - val_loss: 0.0056 - val_accuracy: 0.4589
Epoch 16/100
137/137 [==============================] - 4s 31ms/step - loss: 0.0062 - accuracy: 0.3476 - val_loss: 0.0056 - val_accuracy: 0.4616
Epoch 17/100
136/137 [============================>.] - ETA: 0s - loss: 0.0062 - accuracy: 0.3605WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 4s 30ms/step - loss: 0.0062 - accuracy: 0.3609 - val_loss: 0.0056 - val_accuracy: 0.4643
Epoch 18/100
137/137 [==============================] - 11s 79ms/step - loss: 0.0062 - accuracy: 0.3561 - val_loss: 0.0056 - val_accuracy: 0.4670
Epoch 19/100
137/137 [==============================] - 5s 34ms/step - loss: 0.0062 - accuracy: 0.3631 - val_loss: 0.0056 - val_accuracy: 0.4616
Epoch 20/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0062 - accuracy: 0.3517 - val_loss: 0.0056 - val_accuracy: 0.4670
Epoch 21/100
137/137 [==============================] - ETA: 0s - loss: 0.0062 - accuracy: 0.3547WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 4s 31ms/step - loss: 0.0062 - accuracy: 0.3547 - val_loss: 0.0056 - val_accuracy: 0.4643
Epoch 22/100
137/137 [==============================] - 4s 31ms/step - loss: 0.0062 - accuracy: 0.3599 - val_loss: 0.0056 - val_accuracy: 0.4616
Epoch 23/100
137/137 [==============================] - 4s 30ms/step - loss: 0.0062 - accuracy: 0.3629 - val_loss: 0.0056 - val_accuracy: 0.4634
Epoch 24/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0062 - accuracy: 0.3615 - val_loss: 0.0056 - val_accuracy: 0.4652
Epoch 25/100
137/137 [==============================] - 4s 31ms/step - loss: 0.0061 - accuracy: 0.3698 - val_loss: 0.0056 - val_accuracy: 0.4634
Epoch 26/100
135/137 [============================>.] - ETA: 0s - loss: 0.0062 - accuracy: 0.3653WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 4s 32ms/step - loss: 0.0062 - accuracy: 0.3645 - val_loss: 0.0056 - val_accuracy: 0.4616
Epoch 27/100
137/137 [==============================] - 5s 37ms/step - loss: 0.0062 - accuracy: 0.3604 - val_loss: 0.0056 - val_accuracy: 0.4661
Epoch 28/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0061 - accuracy: 0.3743 - val_loss: 0.0056 - val_accuracy: 0.4589
Epoch 29/100
137/137 [==============================] - 11s 79ms/step - loss: 0.0061 - accuracy: 0.3682 - val_loss: 0.0056 - val_accuracy: 0.4688
Epoch 30/100
135/137 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.3727WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 4s 32ms/step - loss: 0.0062 - accuracy: 0.3702 - val_loss: 0.0056 - val_accuracy: 0.4616
Epoch 31/100
137/137 [==============================] - 5s 34ms/step - loss: 0.0061 - accuracy: 0.3727 - val_loss: 0.0056 - val_accuracy: 0.4625
Epoch 32/100
137/137 [==============================] - 4s 30ms/step - loss: 0.0061 - accuracy: 0.3627 - val_loss: 0.0056 - val_accuracy: 0.4652
Epoch 33/100
137/137 [==============================] - 5s 36ms/step - loss: 0.0061 - accuracy: 0.3739 - val_loss: 0.0056 - val_accuracy: 0.4580
Epoch 34/100
137/137 [==============================] - 5s 33ms/step - loss: 0.0061 - accuracy: 0.3725 - val_loss: 0.0056 - val_accuracy: 0.4607
Epoch 35/100
137/137 [==============================] - 4s 28ms/step - loss: 0.0061 - accuracy: 0.3672 - val_loss: 0.0056 - val_accuracy: 0.4661
Epoch 36/100
137/137 [==============================] - 4s 33ms/step - loss: 0.0061 - accuracy: 0.3716 - val_loss: 0.0056 - val_accuracy: 0.457

137/137 [==============================] - 4s 28ms/step - loss: 0.0061 - accuracy: 0.3734 - val_loss: 0.0056 - val_accuracy: 0.4554
Epoch 40/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0061 - accuracy: 0.3707 - val_loss: 0.0057 - val_accuracy: 0.4616
Epoch 41/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0061 - accuracy: 0.3668 - val_loss: 0.0057 - val_accuracy: 0.4625
Epoch 42/100
137/137 [==============================] - 5s 36ms/step - loss: 0.0061 - accuracy: 0.3720 - val_loss: 0.0056 - val_accuracy: 0.4607
Epoch 43/100
136/137 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.3725WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 5s 34ms/step - loss: 0.0061 - accuracy: 0.3734 - val_loss: 0.0057 - val_accuracy: 0.4518
Epoch 44/100
137/137 [==============================] - 5s 34ms/step - loss: 0.0061 - accuracy: 0.3698 - val_loss: 0.0056 - val_accuracy: 0.4598
Epoch 45/100
137/137 [==============================] - 5s 34ms/step - loss: 0.0061 - accuracy: 0.3688 - val_loss: 0.0057 - val_accuracy: 0.4563
Epoch 46/100
137/137 [==============================] - 5s 34ms/step - loss: 0.0061 - accuracy: 0.3739 - val_loss: 0.0057 - val_accuracy: 0.4598
Epoch 47/100
137/137 [==============================] - 4s 33ms/step - loss: 0.0060 - accuracy: 0.3814 - val_loss: 0.0056 - val_accuracy: 0.4643
Epoch 48/100
136/137 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.3752WARNING:tensorflow:Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).


137/137 [==============================] - 5s 35ms/step - loss: 0.0061 - accuracy: 0.3743 - val_loss: 0.0056 - val_accuracy: 0.4625
Epoch 49/100
137/137 [==============================] - 5s 35ms/step - loss: 0.0061 - accuracy: 0.3750 - val_loss: 0.0056 - val_accuracy: 0.4607


In [ ]:
items = {}
_numberOfVisits = 0
for curr_doc in _docs[:cut_index-1]:
    if curr_doc[2] not in items:
        items[curr_doc[2]] = ItemState()
    next_doc = _docs[_docs.index(curr_doc)+1]
    items[curr_doc[2]].addVisitToItem(next_doc[2], 1000, next_doc[0])
    items[curr_doc[2]].updateVisits(_numberOfVisits)
    _numberOfVisits+=1

In [116]:
# get frency recence
crfWeights = [ItemFactory() for i in range(cut_index-10)]

for i in range(cut_index-10,len(_docs)):
    item_factory = ItemFactory()
    for _doc in _docs[:i]:
        _id = _doc[2]
        _time = '00:10:01'
        item_factory.update(_id, _time)
    crfWeights.append(item_factory)

In [117]:
print(len(crfWeights))

5460


In [ ]:
#Check data
#for item,item_state in items.items():
#    print(item,':::', item_state.markovDescription())

# Markov chain
#for curr_doc in _docs[cut_index:cut_index+1]:
#    rec_item = []
#    prev_doc = _docs[_docs.index(curr_doc)-1]
#    print(curr_doc[2])
#    for _item,_state in items[prev_doc[2]].nextVisits.items():
#        xn = items[prev_doc[2]].numberOfVisits
#        x = len(_state)
#        markov = (x + 1) / (xn + 1)
#        rec_item.append((_item, markov))
#    sorted_rec_item = sorted(rec_item, key=takeSecond, reverse=True)
#    for t in sorted_rec_item[:10]:
#        print('\t',str(sorted_rec_item.index(t)+1)+')',t[0][0:50],t[1])

In [164]:
l = 2.5
d = 0.07
g = 1
#access rank
precision_1_a = []
precision_2_a = []
precision_3_a = []
precision_4_a = []
precision_5_a = []
mrr_a = []
#routine based
precision_1_r = []
precision_2_r = []
precision_3_r = []
precision_4_r = []
precision_5_r = []
mrr_r = []
#markov
precision_1_m = []
precision_2_m = []
precision_3_m = []
precision_4_m = []
precision_5_m = []
mrr_m = []
#frequency recency
precision_1_f = []
precision_2_f = []
precision_3_f = []
precision_4_f = []
precision_5_f = []
mrr_f = []

items = {}
_numberOfVisits = 0
for curr_doc in _docs[:cut_index-1]:
    if curr_doc[2] not in items:
        items[curr_doc[2]] = ItemState()
    next_doc = _docs[_docs.index(curr_doc)+1]
    items[curr_doc[2]].addVisitToItem(next_doc[2], 1000, next_doc[0])
    items[curr_doc[2]].updateVisits(_numberOfVisits)
    _numberOfVisits+=1

    
#all ids
all_ids = defaultdict(int)
all_apps = defaultdict(int)
print(cut_index)
for _doc in _docs[:cut_index]:
    all_ids[_doc[2]] +=1
    
pred_i = 0
for curr_doc in _docs[cut_index:cut_index+n_tests-1]:
    if curr_doc[2] in items:
        print(curr_doc[2],'num visits',items[curr_doc[2]].numberOfVisits,pred_i)
    else:
        print(curr_doc[2],'num visits','0',pred_i)
    #markov
    prev_doc = _docs[_docs.index(curr_doc)-1]
    markovs = {}
    if prev_doc[2] in items:
        for _item,_state in items[prev_doc[2]].nextVisits.items():
            xn = items[prev_doc[2]].numberOfVisits
            x = len(_state)
            markov = (x + 1) / (xn + 1)
            markovs[_item] = markov
    
    #updating markov
    if curr_doc[2] not in items:
        items[curr_doc[2]] = ItemState()
    items[prev_doc[2]].addVisitToItem(curr_doc[2], 1000, curr_doc[0])
    items[prev_doc[2]].updateVisits(_numberOfVisits)
    _numberOfVisits+=1
    
    #markov
    rec_kw = []
    for _id,v in markovs.items():
        markov = markovs[_id]
        rec_kw.append((_id,markov))
    sorted_terms_bl = sorted(rec_kw, key=takeSecond, reverse=True)
    
    recommended_docs_m = []
    for t in sorted_terms_bl:
        #print('\t',str(sorted_terms_bl.index(t)+1)+')',t[0][0:50],t[1])
        recommended_docs_m.append(t[0][:12])
        
    #frequency recency
    rec_kw = []
    for _id,v in markovs.items():
        crf = crfWeights[cut_index+pred_i].items[_id].crfWeight
        rec_kw.append((_id,crf))
    sorted_terms_bl = sorted(rec_kw, key=takeSecond, reverse=True)
    
    recommended_docs_f = []
    for t in sorted_terms_bl:
        #print('\t',str(sorted_terms_bl.index(t)+1)+')',t[0][0:50],t[1])
        recommended_docs_f.append(t[0][:12])
    
    #access rank
    rec_kw = []
    for _id,v in all_ids.items():
        markov = markovs[_id] if _id in markovs else 1e-4
        crf = crfWeights[cut_index+pred_i].items[_id].crfWeight
        rec_kw.append((_id,pow(markov,l)*pow(crf,1/l)))
    sorted_terms_bl = sorted(rec_kw, key=takeSecond, reverse=True)
    
    recommended_docs_a = []
    for t in sorted_terms_bl:
        #print('\t',str(sorted_terms_bl.index(t)+1)+')',t[0][0:50],t[1])
        recommended_docs_a.append(t[0][:12])
        
    #routine
    rec_kw = []
    pattern_entities = {}
    _tmp_patterns = []
    for i in _pred_patterns[pred_i]:
        pattern_words = particle.per_topic_word_counts[i[0]]
        tmp_words = []
        normalized_score = []
        for word in pattern_words:
            if 'title_' not in word:
                continue
            _tmp_patterns.append((word,(ln(pattern_words[word] + particle.theta_0[0]))*i[1]))
    flat_tmp_patterns = flatRecom(_tmp_patterns)
    pattern_entities = {flat_tmp_patterns[i][0]: flat_tmp_patterns[i][1] for i in range(len(flat_tmp_patterns))}
      
    for _id,v in all_ids.items():
        markov = markovs[_id] if _id in markovs else 1e-4
        routine = pattern_entities[_id] if _id in pattern_entities else 1e-20
        crf = crfWeights[cut_index+pred_i].items[_id].crfWeight
#        rec_kw.append((_id,pow(markov,l)*pow(routine,g)*pow(crf,1/d)))
        rec_kw.append((_id,pow(routine,g)))
    
    sorted_terms_bl = sorted(rec_kw, key=takeSecond, reverse=True)
    
    recommended_docs_r = []
    for t in sorted_terms_bl:
        #print('\t',str(sorted_terms_bl.index(t)+1)+')',t[0][0:50],t[1])
        recommended_docs_r.append(t[0][:12])
    
    pred_i +=1
    
    # Evaluation
    future_doc = curr_doc[2][:12]
    all_apps[curr_doc[1]] +=1
    
    if future_doc not in [k[:12] for k in all_ids.keys()]:
        continue
    #if _hit[pred_i-1] == 0:
    #    continue
    
    # markov
    correct_doc = []
    if future_doc in recommended_docs_m[:1]:
        correct_doc.append(future_doc)
    precision_1_m.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_m[:2]:
        correct_doc.append(future_doc)
    precision_2_m.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_m[:3]:
        correct_doc.append(future_doc)
    precision_3_m.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_m[:4]:
        correct_doc.append(future_doc)
    precision_4_m.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_m[:5]:
        correct_doc.append(future_doc)
    precision_5_m.append(len(correct_doc))
    
    if future_doc in recommended_docs_m:
        mrr_m.append(1/(recommended_docs_m.index(future_doc)+1))
    else:
        mrr_m.append(0)
        
    # frequency recency
    correct_doc = []
    if future_doc in recommended_docs_f[:1]:
        correct_doc.append(future_doc)
    precision_1_f.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_f[:2]:
        correct_doc.append(future_doc)
    precision_2_f.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_f[:3]:
        correct_doc.append(future_doc)
    precision_3_f.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_f[:4]:
        correct_doc.append(future_doc)
    precision_4_f.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_f[:5]:
        correct_doc.append(future_doc)
    precision_5_f.append(len(correct_doc))
    
    if future_doc in recommended_docs_f:
        mrr_f.append(1/(recommended_docs_f.index(future_doc)+1))
    else:
        mrr_f.append(0)
    
    # access rank
    correct_doc = []
    if future_doc in recommended_docs_a[:1]:
        correct_doc.append(future_doc)
    precision_1_a.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_a[:2]:
        correct_doc.append(future_doc)
    precision_2_a.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_a[:3]:
        correct_doc.append(future_doc)
    precision_3_a.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_a[:4]:
        correct_doc.append(future_doc)
    precision_4_a.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_a[:5]:
        correct_doc.append(future_doc)
    precision_5_a.append(len(correct_doc))
    
    if future_doc in recommended_docs_a:
        mrr_a.append(1/(recommended_docs_a.index(future_doc)+1))
    else:
        mrr_a.append(0)
    
    
    correct_doc = []
    if future_doc in recommended_docs_r[:1]:
        correct_doc.append(future_doc)
    precision_1_r.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_r[:2]:
        correct_doc.append(future_doc)
    precision_2_r.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_r[:3]:
        correct_doc.append(future_doc)
    precision_3_r.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_r[:4]:
        correct_doc.append(future_doc)
    precision_4_r.append(len(correct_doc))
    correct_doc = []
    if future_doc in recommended_docs_r[:5]:
        correct_doc.append(future_doc)
    precision_5_r.append(len(correct_doc))
    
    if future_doc in recommended_docs_r:
        mrr_r.append(1/(recommended_docs_r.index(future_doc)+1))
    else:
        mrr_r.append(0)


print('**Makov')
print('Hitrate@1: ', np.mean(precision_1_m))
print('Hitrate@2: ', np.mean(precision_2_m))
print('Hitrate@3: ', np.mean(precision_3_m))
print('Hitrate@4: ', np.mean(precision_4_m))
print('Hitrate@5: ', np.mean(precision_5_m))
print('MRR: ', np.mean(mrr_m))

print('**Frequency recency')
print('Hitrate@1: ', np.mean(precision_1_f))
print('Hitrate@2: ', np.mean(precision_2_f))
print('Hitrate@3: ', np.mean(precision_3_f))
print('Hitrate@4: ', np.mean(precision_4_f))
print('Hitrate@5: ', np.mean(precision_5_f))
print('MRR: ', np.mean(mrr_f))

print('**Access rank')
print('Hitrate@1: ', np.mean(precision_1_a))
print('Hitrate@2: ', np.mean(precision_2_a))
print('Hitrate@3: ', np.mean(precision_3_a))
print('Hitrate@4: ', np.mean(precision_4_a))
print('Hitrate@5: ', np.mean(precision_5_a))
print('MRR: ', np.mean(mrr_a))

print('**Routine')
print('Hitrate@1: ', np.mean(precision_1_r))
print('Hitrate@2: ', np.mean(precision_2_r))
print('Hitrate@3: ', np.mean(precision_3_r))
print('Hitrate@4: ', np.mean(precision_4_r))
print('Hitrate@5: ', np.mean(precision_5_r))
print('MRR: ', np.mean(mrr_r))

4352
title_elements.css num visits 24 0
title_medisyn_|_prototype num visits 246 1
title_userauthen.js num visits 43 2
title_medisyn_|_prototype num visits 247 3
title_chenhe_—_chenhe@shell:_~_—_-bash_—_80×36 num visits 0 4
title_ver4mongodb_—_mongod_◂_sudo_—_80×24 num visits 9 5
title_ver7anno_—_-bash_—_80×24 num visits 2 6
title_userauthen.js_routes num visits 6 7
title_ver7anno_—_-bash_—_80×24 num visits 3 8
title_userauthen.js_routes num visits 7 9
title_ver7anno_—_node_bin/www_—_80×24 num visits 29 10
title_medisyn_|_prototype num visits 248 11
title_userauthen.js num visits 44 12
title_medisyn_|_prototype num visits 249 13
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf num visits 89 14
title_medisyn_|_prototype num visits 250 15
title_elements.css num visits 25 16
title_userauthen.js num visits 45 17
title_medisyn_|_prototype num visits 251 18
title_userauthen.js num visits 46 19
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf 

title_welcome_to_the_cvast_homepage!_|_centre_for_visual_analytics_science_and_technology num visits 0 176
title_luana.micallef_chat num visits 57 177
title_downloads num visits 25 178
title_welcome_to_the_cvast_homepage!_|_centre_for_visual_analytics_science_and_technology num visits 1 179
title_tacit_knowledge_-_google_search num visits 0 180
title_tacit_knowledge_-_wikipedia num visits 0 181
title_downloads num visits 26 182
title_google_drive num visits 21 183
title_myprez num visits 6 184
title_visual_analytics num visits 0 185
title_a_case_study_using_visuali... num visits 35 186
title_2.pdf num visits 5 187
title_visual_analytics num visits 1 188
title_idv_course_2018_-_google_docs num visits 30 189
title_weboodi num visits 11 190
title_idv_course_2018_-_google_docs num visits 31 191
title_idv_course_2018 num visits 32 192
title_https://docs.google.com/presentation/u/0/d/1o2xt1vsqnmpgtpxaudv_fqr3cmv_finvql-lix6oie8/edit?usp=drive_web num visits 0 193
title_idv_course_2018 num vi

title_new_message_compose num visits 19 334
title_idv_course_2018_-_google_docs num visits 54 335
title_new_message_compose num visits 20 336
title_oswald_barral_defends_his_phd_thesis_view num visits 2 337
title_new_message_compose num visits 21 338
title_idv_course_structure_compose num visits 0 339
title_google_docs_-_google_search num visits 5 340
title_idv_course_2018_-_google_docs num visits 55 341
title_oswald_barral_defends_his_phd_thesis_view num visits 3 342
title_idv_course_structure_view num visits 0 343
title_idv_course_2018_-_google_docs num visits 56 344
title_idv_course_structure_view num visits 1 345
title_idv_course_2018_-_google_docs num visits 57 346
title_google_docs_-_google_search num visits 6 347
title_oswald_barral_defends_his_phd_thesis_view num visits 4 348
title_google_docs_-_google_search num visits 7 349
title_oswald_barral_defends_his_phd_thesis_view num visits 5 350
title_google_docs_-_google_search num visits 8 351
title_google_docs num visits 12 352
ti

title_defining_and_concept_ualizi num visits 3 488
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf num visits 101 489
title_luana.micallef_chat num visits 72 490
title_inbox_view num visits 39 491
title_oswald_barral_defends_his_phd_thesis_view num visits 11 492
title_luana.micallef_chat num visits 73 493
title_web.pdf num visits 4 494
title_defining_and_concept_ualizi num visits 4 495
title_luana.micallef_chat num visits 74 496
title_defining_and_concept_ualizi num visits 5 497
title_insight_provenance_for_vast_view num visits 15 498
title_idv_course_structure_view num visits 2 499
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf num visits 102 500
title_defining_and_concept_ualizi num visits 6 501
title_oswald_barral_defends_his_phd_thesis_view num visits 12 502
title_defining_and_concept_ualizi num visits 7 503
title_unisport num visits 8 504
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf num visits 103 50

title_history_evaluation num visits 63 663
title_luana.micallef_chat num visits 79 664
title_insight_provenance_for_vast_view num visits 17 665
title_medisyn_questionnaire_-_google_forms num visits 60 666
title_luana.micallef_chat num visits 80 667
title_interactive_recommender_systems:_a_survey_of_the_state_of_the_art_and_future_research_challenges_and_opportunities num visits 0 668
title_medisyn_questionnaire_-_google_forms num visits 61 669
title_the_role_of_explicit_knowledge:_a_conceptual_model_of_knowledge-assisted_visual_analytics num visits 4 670
title_history_evaluation num visits 64 671
title_how_to_destroy_a_javascript_object?_-_stack_overflow num visits 1 672
title_luana.micallef_chat num visits 81 673
title_how_to_destroy_a_javascript_object?_-_stack_overflow num visits 2 674
title_the_role_of_explicit_knowledge:_a_conceptual_model_of_knowledge-assisted_visual_analytics num visits 5 675
title_how_progressive_visualizations_affect_exploratory_analysis_-_google_search num vi

title_history.js num visits 39 827
title_history.html num visits 24 828
title_find_results num visits 1 829
title_history.js num visits 40 830
title_history.html num visits 25 831
title_medisyn_|_prototype num visits 328 832
title_lidwell,_holden,_butler_-_2010_-_universal_principles_of_design.pdf num visits 118 833
title_web.pdf num visits 19 834
title_history.html num visits 26 835
title_history_evaluation num visits 71 836
title_oswald_barral_defends_his_phd_thesis_view num visits 14 837
title_your_application_in_enter_finland_is_waiting_to_be_processed_view num visits 0 838
title_web.pdf num visits 20 839
title_history_evaluation num visits 72 840
title_medisyn_|_history num visits 182 841
title_history_evaluation num visits 73 842
title_https://d4health.hiit.fi/retrieveallhis num visits 75 843
title_medisyn_|_prototype num visits 329 844
title_slack_-_ubiquitous_interaction_(uix)_group num visits 11 845
title_medisyn_|_prototype num visits 330 846
title_ver7anno_—_node_bin/www_—_8

title_medisyn_|_prototype num visits 336 980
title_new_tab num visits 59 981
title_medisyn_|_prototype num visits 337 982
title_medisyn_|_history num visits 183 983
title_medisyn_|_prototype num visits 338 984
title_luana.micallef_call num visits 21 985
title_medisyn_|_prototype num visits 339 986
title_online_user_study_with_movie_tickets_as_rewards_view num visits 78 987
title_calendar num visits 20 988
title_online_user_study_with_movie_tickets_as_rewards_view num visits 79 989
title_invitation_to_design_critique_view num visits 0 990
title_invitation_to_design_critique_compose num visits 0 991
title_invitation_to_design_critique_view num visits 1 992
title_examples_of_online_studies_view num visits 10 993
title_invitation_to_design_critique_view num visits 2 994
title_new_message_compose num visits 23 995
title_insight_provenance_for_vast_view num visits 21 996
title_insight_provenance_for_vast_compose num visits 24 997
title_history_evaluation num visits 75 998
**Makov
Hitrate@1: 